In [3]:
from typing import Annotated, Literal, Optional
from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages



def update_dialog_stack(left: list[str], right: Optional[str]) -> list[str]:
    """Push or pop the state."""
    if right is None:
        return left
    if right == "pop":
        return left[:-1]
    return left + [right]

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: str
    dialog_state: Annotated[
            list[
                Literal[
                    "assistant",
                    "query",
                    "requset",
                ]
            ],
            update_dialog_stack,
        ]
    order_id: int = None # int형에 None으로 초기화해도 되나?
    confirmation: bool = False

In [4]:
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.messages import AIMessage

from products.models import Order
from django.db.models import QuerySet

class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)

            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break

        if isinstance(result, Order):
            result = result.to_dict()
            result = AIMessage(content=result)
            # result = json.dumps(result)
        elif isinstance(result, QuerySet):
            result = [order.to_dict() for order in result]
            result = AIMessage(content=result)
        # print("assistant 출력\n", type(result))
        return {"messages": result}

ModuleNotFoundError: No module named 'products'

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

order_request_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문 요청 처리를 담당하는 주문봇이야.
            사용자 요청이 주문 변경, 주문 취소와 관련된 경우, 

            Current user ID: {user_info}
            """
       ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm =  ChatOpenAI()

order_request_runnable = order_request_prompt | llm

order_request_runnable

SyntaxError: invalid syntax (1018811084.py, line 1)

주문 내역 출력

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    ("system", 
     "너는 사용자에게 변경할 주문을 묻는 주문봇이야"
     "사용자의 주문 내역을 보여주고 그 가운데 변경할 주문을 선택해달라는 메시지를 작성해줘"
     "사용자 주문 내역: {orders}"),
     MessagesPlaceholder(variable_name="messgaes")
)
llm =  ChatOpenAI()
manage_order_runnable = llm | prompt 

In [ ]:
from chain.langgraph_tools import fetch_recent_order

def display_user_order(state: State):
    user_id = State["user_id"]
    recent_orders = fetch_recent_order(user_id)
    response = manage_order_runnable.invoke(**state, {"orders": recent_orders})
    return {"orders":response}


선택했으면 주문 변경 진행
* 진행 전에 사용자 개입
---
* 이전 노드 출력에서 Dict에 할당한 key를 그 다음 노드에서 접근하려면?
* order_id에 담아 다음 노드로 전달

In [ ]:
def manage_order_route(state: State) -> Literal["order_change", "order_cancel"]:
    order_id = state["order_id"]
    messages = state['messages']
    last_message = messages[-1]
    tool_calls = last_message.tool_calls
    if order_id is None:
        return "display_user_order"
    if tool_calls[0]["name"] == "order_change":
        return "order_change"
    if tool_calls[0]["name"] == "order_cancel":
        return "order_cancel"
    # 이후에 leave_skill 등 추가


In [ ]:
def route_order_request(state: State):
    order_id = state["order_id"]
    if order_id is None:
        return "display_user_order"


# Define Graph

In [ ]:
from langgraph.graph import StateGraph
builder = StateGraph(State)
builder.add_node("order_request", Assistant(order_request_runnable))
builder.add_node("display_user_order", display_user_order)
builder.add_node("change_order", chage_order)
builder.add_node("cancel_order", cancel_order)

In [ ]:
builder.add_conditional_edges(
    "order_request",
    orer_change_route,
    {
        "order_change": "order_change",
        "enter_order_request": "enter_order_request",
        "primary_assistant_tools": "primary_assistant_tools",
        END: END,
    },
)